


# 1. IMPORTANDO BIBLIOTECAS



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
from google.colab import drive

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import sys
print(sys.getrecursionlimit())

import warnings
warnings.filterwarnings("ignore")


1000


In [ ]:
sys.setrecursionlimit(10000)

# 2 CARREGANDO AS BASES

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
metavg = pd.read_json('/content/gdrive/MyDrive/DATABASES/meta_Video_Games.json', lines=True)

In [ ]:
videogame = pd.read_json('/content/gdrive/MyDrive/DATABASES/Video_Games_sample.json',lines=True)

# 3.TRATAMENTO E LIMPEZA DOS DADOS




In [ ]:
metavg.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [ ]:
videogame.head()

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified,vote,image
0,1,"01 2, 2018",A16FEXIKAPT24U,B00000JRSB,{'Format:': ' Video Game'},Nicholas Sabin,Game disc was cracked thats pretty lame,Game disc 1 was cracked,1514851200,NaN,NaN,NaN
1,5,"01 2, 2018",A3ISBEPYLY8IMO,B00000JRSB,{'Format:': ' Video Game'},Jimmy,Received this in perfect condition. Great rese...,Received this in perfect condition. Great rese...,1514851200,1.0,NaN,NaN
2,5,"12 21, 2017",A1RIUB1PZWLOVZ,B00000JRSB,{'Format:': ' Video Game'},Amazon Customer,I had a hard time finding this game locally or...,I am very pleased with this purchase,1513814400,1.0,NaN,NaN
3,5,"12 20, 2017",A2VY0K54SOCG0F,B00000JRSB,{'Format:': ' Video Game'},Oscentatious,This game is just as amazing as the first time...,This is the best deal for this game,1513728000,0.0,NaN,NaN
4,5,"12 19, 2017",A1ABJQ2REV14OQ,B00000JRSB,{'Format:': ' Video Game'},Thiago,"I bought as a gift for my friend, she is so ex...",Loved by lots of people,1513641600,1.0,NaN,NaN


In [ ]:
metavg.dtypes

category           object
tech1              object
description        object
fit                object
title              object
also_buy           object
tech2              object
brand              object
feature            object
rank               object
also_view          object
main_cat           object
similar_item       object
date               object
price              object
asin               object
imageURL           object
imageURLHighRes    object
details            object
dtype: object

In [ ]:
metavg.isnull().sum()

category             0
tech1                0
description          0
fit                  0
title                0
also_buy             0
tech2                0
brand                0
feature              0
rank                 0
also_view            0
main_cat             0
similar_item         0
date                 0
price                0
asin                 0
imageURL             0
imageURLHighRes      0
details            107
dtype: int64

In [ ]:
videogame.dtypes

overall             int64
reviewTime         object
reviewerID         object
asin               object
style              object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
verified          float64
vote               object
image              object
dtype: object

In [ ]:
videogame.isnull().sum()

overall                0
reviewTime             0
reviewerID             0
asin                   0
style             135402
reviewerName          51
reviewText           345
summary              166
unixReviewTime         0
verified           69016
vote              396439
image             448911
dtype: int64

In [ ]:
videogame['reviewText'].fillna('Not review', inplace=True)
videogame['reviewerName'].fillna('No reviewer', inplace=True)
videogame['summary'].fillna('No summary', inplace=True)

In [ ]:
videogame.isnull().sum()

overall                0
reviewTime             0
reviewerID             0
asin                   0
style             135402
reviewerName           0
reviewText             0
summary                0
unixReviewTime         0
verified           69016
vote              396439
image             448911
dtype: int64

# Fazendo backups

In [ ]:
backup = videogame.copy()

In [ ]:
videogame = backup.copy()

# 3. IMPLEMENTANDO O MODELO DE CLASSIFICAÇÃO

In [ ]:
videogame.drop(columns=['vote','image',],inplace=True)

In [ ]:
# o modelo ira preencher as colunas style verified
dfclass = videogame[['overall','verified','unixReviewTime']].dropna()

In [ ]:
X = dfclass.drop(columns=['verified'])
Y = dfclass[['verified']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 3.1 KNN

In [ ]:
 resultados = {'' : ['KNN','Random Forest','Logistic Regression', 'XgBoosting'],
               'Accuracy' : [],
               'Precision' : [],
               'Recall' : [],
               'F1-Score' : [],}

In [ ]:
params = {
    'n_neighbors' : [5, 6, 7, 10]
}

clf = GridSearchCV(KNeighborsClassifier(), param_grid=params, scoring='f1', cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [5, 6, 7, 10]}, scoring='f1')

In [ ]:
y_pred = clf.predict(X_test)

resultados['Accuracy'].append(accuracy_score(y_test, y_pred))
resultados['Precision'].append(precision_score(y_test, y_pred))
resultados['Recall'].append(recall_score(y_test, y_pred))
resultados['F1-Score'].append(f1_score(y_test, y_pred))


In [ ]:
clf_knn=clf.best_estimator_

#3.2 Random Forest

In [ ]:
params = {
    'max_depth' : [2,5,7],
    'n_estimators' : [100,200,300],
    'class_weight' : [None,'balanced']
}

clf = GridSearchCV(RandomForestClassifier(), param_grid=params, scoring='f1',cv=5)

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'max_depth': [2, 5, 7],
                         'n_estimators': [100, 200, 300]},
             scoring='f1')

In [ ]:
clf_rf = clf.best_estimator_

# 3.3 Logistic Regression

In [ ]:
params = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.01, 0.1, 1],
    'class_weight' : [None, 'balanced']
}

clf = GridSearchCV(LogisticRegression(), param_grid=params, scoring='f1',cv=5)

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [ ]:
y_pred = clf.predict(X_test)

resultados['Accuracy'].append(accuracy_score(y_test, y_pred))
resultados['Precision'].append(precision_score(y_test, y_pred))
resultados['Recall'].append(recall_score(y_test, y_pred))
resultados['F1-Score'].append(f1_score(y_test, y_pred))

In [ ]:
clf_lg = clf.best_estimator_

# 3.4 XgBoosting

In [ ]:
params = {
    'max_depth' : [6,7,8,9],
    'gamma' : [0, 0.01, 0.1]
    
}

clf = GridSearchCV(XGBClassifier(), param_grid=params, scoring='f1',cv=5)

clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid={'gamma': [0, 0.01, 0.1], 'max_depth': [6, 7, 8, 9]},
             scoring='f1')

In [ ]:
y_pred = clf.predict(X_test)

resultados['Accuracy'].append(accuracy_score(y_test, y_pred))
resultados['Precision'].append(precision_score(y_test, y_pred))
resultados['Recall'].append(recall_score(y_test, y_pred))
resultados['F1-Score'].append(f1_score(y_test, y_pred))


In [ ]:
clf_lg = clf.best_estimator_


In [ ]:
print(resultados)

{'': ['KNN', 'Random Forest', 'Logistic Regression', 'XgBoosting'], 'Accuracy': [0.8513847354050188, 0.8162397607593291, 0.8609153556104537], 'Precision': [0.8695145295700859, 0.8162397607593291, 0.8675681779684942], 'Recall': [0.9623428962836708, 1.0, 0.9790528378227695], 'F1-Score': [0.9135766997338496, 0.8988237989218754, 0.9199452181917513]}


# 4.1 O RECALL NO RANDOM FOREST FOI MAIOR PORTANTO O USAREMOS.

In [ ]:
dfnan = videogame[videogame['verified'].isnull()].reset_index().drop(columns=['index'])

In [ ]:
X = dfnan.drop(columns=['verified', 'reviewTime', 'reviewerID','asin','style','reviewerName','reviewText','summary'])

In [ ]:
X.head()

,overall,unixReviewTime
0,1,1514851200
1,5,1503878400
2,5,1502150400
3,5,1495497600
4,5,1492473600


In [ ]:
dfnan['verified'] = clf_rf.predict(X)

In [ ]:
videogame.dropna(subset=['verified'],inplace=True)

In [ ]:
videogame.isnull().sum()

overall                0
reviewTime             0
reviewerID             0
asin                   0
style             114843
reviewerName           0
reviewText             0
summary                0
unixReviewTime         0
verified               0
dtype: int64

In [ ]:
backup.isnull().sum()

overall                0
reviewTime             0
reviewerID             0
asin                   0
style             135402
reviewerName           0
reviewText             0
summary                0
unixReviewTime         0
verified           69016
vote              396439
image             448911
dtype: int64

In [ ]:
dfinal = pd.concat([videogame.reset_index().drop(columns=['index']),dfnan.reset_index().drop(columns=['index'])])

In [ ]:
dfinal.head()

,overall,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,verified
0,5,"01 2, 2018",A3ISBEPYLY8IMO,B00000JRSB,{'Format:': ' Video Game'},Jimmy,Received this in perfect condition. Great rese...,Received this in perfect condition. Great rese...,1514851200,1.0
1,5,"12 21, 2017",A1RIUB1PZWLOVZ,B00000JRSB,{'Format:': ' Video Game'},Amazon Customer,I had a hard time finding this game locally or...,I am very pleased with this purchase,1513814400,1.0
2,5,"12 20, 2017",A2VY0K54SOCG0F,B00000JRSB,{'Format:': ' Video Game'},Oscentatious,This game is just as amazing as the first time...,This is the best deal for this game,1513728000,0.0
3,5,"12 19, 2017",A1ABJQ2REV14OQ,B00000JRSB,{'Format:': ' Video Game'},Thiago,"I bought as a gift for my friend, she is so ex...",Loved by lots of people,1513641600,1.0
4,5,"12 9, 2017",A368DF2G0T502X,B00000JRSB,{'Format:': ' Video Game'},Jonathan carter,"Looks good, some scratches on discs but plays.",Five Stars,1512777600,1.0


# 4.2 RECOMENDAÇÃO

In [ ]:
dprodutos = dfinal.groupby(by='asin', as_index=False) \
          .agg({'reviewerID' : pd.Series.nunique}) \
          .rename(columns = {'reviewerID' : 'Count_reviewerID'}) \
          .set_index('asin') \
          .sort_values('Count_reviewerID', ascending=False)

In [ ]:
dprodutos.head(10)

,Count_reviewerID
asin,
B00HTK1NCS,5511
B004RMK57U,4263
B00178630A,3731
B00KKAQYXM,3675
B00JJNQG98,3364
B003ZSP0WW,3362
B00E4MQODC,2967
B000B9RI14,2808
B00DBDPOZ4,2715


In [ ]:
dvendas = dprodutos.head(10).index.tolist()

In [ ]:
dvendas

['B00HTK1NCS',
 'B004RMK57U',
 'B00178630A',
 'B00KKAQYXM',
 'B00JJNQG98',
 'B003ZSP0WW',
 'B00E4MQODC',
 'B000B9RI14',
 'B00DBDPOZ4',
 'B0009VXBAQ']

CALCULANDO PARA OS VERIFICADOS POR PRODUTO

In [ ]:
dverificado = dfinal.groupby(by='asin', as_index=False) \
            .agg({'verified' : pd.Series.nunique}) \
            .rename(columns = {'verified' : 'verified_count'}) \
            .set_index('asin') \
            .sort_values('verified_count', ascending=False)

In [ ]:
dverificado.head(10)

,verified_count
asin,
B00000JRSB,2
B00J48MUS4,2
B00FLLFJE0,2
B00FNKMVUO,2
B00FWK8UWI,2
B00GANWVJE,2
B00GMFKYK8,2
B00GODZYNA,2
B00GU4F4OM,2


In [ ]:
# pegando o top 2000 verificados
topverificados = dverificado.head(2000).index.tolist()

In [ ]:
topverificados[:10]

['B00000JRSB',
 'B00J48MUS4',
 'B00FLLFJE0',
 'B00FNKMVUO',
 'B00FWK8UWI',
 'B00GANWVJE',
 'B00GMFKYK8',
 'B00GODZYNA',
 'B00GU4F4OM',
 'B00GU8W5AE']

In [ ]:
# fazendo para a base toda
dfinal.groupby(by='asin', as_index=False) \
            .agg({'verified' : pd.Series.nunique}) \
            .rename(columns = {'verified' : 'verified_mean'}).head(20)

,asin,verified_mean
0,B00000JRSB,2
1,B00005NZ1G,2
2,B00005O0I2,2
3,B00005Q8M0,2
4,B00005TNI6,2
5,B000066TS5,2
6,B0000696CZ,2
7,B000087H7T,2
8,B00008J7NZ,2
9,B00009OY9U,2


# 5.0 MATRIZ DE RELACIONAMENTO

In [ ]:
matriz_de_relacionamento_topverificados = dfinal[dfinal['asin'].isin(topverificados)] \
  .pivot_table(index='asin', columns='reviewerID', values='verified').fillna(0)

In [ ]:
matriz_de_relacionamento_topverificados.shape

(266, 315026)

In [ ]:
matriz_de_relacionamento_topverificados.head()

reviewerID,A0003214FKMKJE0PCW3D,A0005622E33Y7RCVKKVN,A00065507CNSR8UHQFCK,A0011756FPL8K71Q5TAQ,A001932810S6RCIFHJ3V,A0025672ALSPPB1RH94H,A00356786CQNSUGGK2FA,A0039616ADOZ0KMWQRNX,A0049794AHFDZTYE50T0,A0059356O50A83G4RYQ7,...,AZZUP6QP2D82V,AZZUQ22H8ZYZU,AZZV78H5A713R,AZZV9M0S13U26,AZZW3Z3NTGMY7,AZZWV8SNTXQ7U,AZZX40NRMUDDQ,AZZX6JJHIO4UZ,AZZYR0PRM4RBT,AZZZOLOP9ZO8B
asin,,,,,,,,,,,,,,,,,,,,,
B00000JRSB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005NZ1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005O0I2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005Q8M0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005TNI6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Quantidade de avaliações
matriz_de_relacionamento_topverificados.sum()

reviewerID
A0003214FKMKJE0PCW3D    1.0
A0005622E33Y7RCVKKVN    3.0
A00065507CNSR8UHQFCK    1.0
A0011756FPL8K71Q5TAQ    1.0
A001932810S6RCIFHJ3V    1.0
                       ... 
AZZWV8SNTXQ7U           1.0
AZZX40NRMUDDQ           1.0
AZZX6JJHIO4UZ           1.0
AZZYR0PRM4RBT           1.0
AZZZOLOP9ZO8B           0.0
Length: 315026, dtype: float64

# 5.1 MATRIZ DE SIMILARIDADES

In [ ]:
cos_asin = cosine_similarity(matriz_de_relacionamento_topverificados)

print(cos_asin)
cos_asin.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


(266, 266)

In [ ]:
# Existe uma similaridades dos produtos, produziremos o modelo de recomendação
def recomendar(asin):
  print(f'Produto pesquisado: {asin}')
  try:
    asin_idx = matriz_de_relacionamento_topverificados.index.tolist().index(asin)
    closest_10_asin = np.argsort(-cos_asin[asin_idx])[1:11]
    print(f'Os 10 produtos mais próximos do asin: {asin} são :')
    return matriz_de_relacionamento_topverificados.index[closest_10_asin].tolist()
  except ValueError as e:
    print(f'{asin} não encontrado na matriz de recomendação. Retornando os 10 produtos mais avaliados')
    return dvendas

TESTANDO OS SISTEMA

In [ ]:
asin = 'B00005Q8M0'
asin_10_mais_proximos = recomendar(asin)

asin_10_mais_proximos

Produto pesquisado: B00005Q8M0
Os 10 produtos mais próximos do asin: B00005Q8M0 são :


['B00005TNI6',
 'B0017KIBAI',
 'B0009VXBAQ',
 'B000FQ2DTA',
 'B0011E4J2Y',
 'B00EZPCX1A',
 'B002TK1PX0',
 'B00000JRSB',
 'B000087H7T',
 'B000FQ9R4E']

In [ ]:
asin = 'asin_inexist'
asin_10_mais_proximos = recomendar(asin)

asin_10_mais_proximos

Produto pesquisado: asin_inexist
asin_inexist não encontrado na matriz de recomendação. Retornando os 10 produtos mais avaliados


['B00HTK1NCS',
 'B004RMK57U',
 'B00178630A',
 'B00KKAQYXM',
 'B00JJNQG98',
 'B003ZSP0WW',
 'B00E4MQODC',
 'B000B9RI14',
 'B00DBDPOZ4',
 'B0009VXBAQ']

# 6.0 CONCLUSÃO

O sistema de recomendação conta com matrizes esparsas, o ideal seria achar uma diminuição, levando a descrição do produto em consideração, avaliando a compra do produto.